# Finetuning GPT-3.5 - CW Coherence.ipynb

Sources: https://blog.futuresmart.ai/fine-tuning-gpt-35-a-step-by-step-guide

In [17]:
# Whether this notebook is first 50% or last 50%
this_nb = 'first_50'


In [18]:
import pandas as pd
import openai


In [19]:

full_human_data = pd.read_csv("full_human_data.csv")
full_human_data.head(5)


,model_task_method,conversation_number,response,coherence_1_incoherent_10_very_coherent
0,td3_cw_direct_prompting_responses,1,Learning to do a handstand is a fun activity f...,1
1,td3_cw_direct_prompting_responses,2,The hawk was used to hunting what he needed fo...,7
2,td3_cw_direct_prompting_responses,3,I love the smell of roasting almonds in the ki...,1
3,td3_cw_direct_prompting_responses,4,Ralph's bedroom was routinely filled up with s...,10
4,td3_cw_direct_prompting_responses,5,Joe had a unique way of dealing with the hospi...,4


In [20]:
# This notebook: train and validate on first 50% of observations for each method
# Get predictions on second 50% of observations for each method
# Keep first 50% of observations for each model_task_method in dataset
# Split on conversation_number LTE 50 and not
if this_nb == 'first_50':
    this_notebook_train_validate = full_human_data[full_human_data['conversation_number'] <= 50]
    this_notebook_predictions = full_human_data[full_human_data['conversation_number'] > 50]
else:
    this_notebook_train_validate = full_human_data[full_human_data['conversation_number'] > 50]
    this_notebook_predictions = full_human_data[full_human_data['conversation_number'] <= 50]


In [21]:
# Print responses for 5 observations
for i in range(5):
    print(this_notebook_train_validate.iloc[i, 2])
    print('')


Learning to do a handstand is a fun activity for people of all ages. It takes practice to master, but once you get the hang of it, you'll learn to balance your body in an upside-down position. It isn't difficult to do a handstand if you just stand on your hands.
John, a space explorer, felt excited when he first stepped on the moon. It was a unique experience that he thought he would never forget. To his surprise, a smell of seared steak followed him up the mountain of dust. It caught him off guard that space smelled of seared steak.

The hawk was used to hunting what he needed for food, but one day he decided to try and be friends with the ground squirrels in the park. He thought it would be a good way to pass the time and to fill his empty belly. He went out of his way to make nice with the squirrels, but they ran away from him. The hawk didn't understand why the ground squirrels didn't want to be his friend.
The hawk was confused, but he soon learned an important lesson. Just like t

### Format Dataset

In [22]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"Score": "' + str(row['coherence_1_incoherent_10_very_coherent']) + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['response'] + "\nYour Task: Rate the coherence of the passage above on a scale of 1 to 10, 1 being incoherent and 10 being very coherent."},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data


In [23]:
# Convert entire dataset
dataset = this_notebook_train_validate
converted_data = convert_to_gpt35_format(dataset)
converted_data[0]['messages']


[{'role': 'user',
  'content': "Learning to do a handstand is a fun activity for people of all ages. It takes practice to master, but once you get the hang of it, you'll learn to balance your body in an upside-down position. It isn't difficult to do a handstand if you just stand on your hands.\nJohn, a space explorer, felt excited when he first stepped on the moon. It was a unique experience that he thought he would never forget. To his surprise, a smell of seared steak followed him up the mountain of dust. It caught him off guard that space smelled of seared steak.\nYour Task: Rate the coherence of the passage above on a scale of 1 to 10, 1 being incoherent and 10 being very coherent."},
 {'role': 'assistant', 'content': '{"Score": "1"}'}]

In [24]:
import json
json.loads(converted_data[0]['messages'][-1]['content'])


{'Score': '1'}

Create Train and Val Set

In [25]:
# Print unique values of model_task_method
this_notebook_train_validate['model_task_method'].unique()


array(['td3_cw_direct_prompting_responses',
       'td3_cw_zero_shot_cot_responses',
       'td3_cw_ape_zero_shot_cot_responses',
       'td3_cw_least_to_most_responses',
       'td3_cw_manual_few_shot_responses', 'td3_cw_manual_cot_responses',
       'td3_cw_tree_of_thought_responses', 'td3_cw_self_refine_responses',
       'gpt4_cw_direct_prompting_responses',
       'gpt4_cw_zero_shot_cot_responses',
       'gpt4_cw_ape_zero_shot_cot_responses',
       'gpt4_cw_tree_of_thought_responses',
       'gpt4_cw_self_refine_responses', 'gpt4_cw_least_to_most_responses',
       'gpt4_cw_manual_few_shot_responses',
       'gpt4_cw_manual_cot_responses'], dtype=object)

In [26]:
from sklearn.model_selection import train_test_split

# Stratified splitting on model_task_method.
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=this_notebook_train_validate['model_task_method'],
    random_state=42  # for reproducibility
)


In [27]:
type(train_data[0])


dict

In [28]:
print(len(train_data))
print(len(val_data))


640
160


Create JSONL file

In [29]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train_" + this_nb + ".jsonl"
validation_file_name = "val_" + this_nb + ".jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)


In [30]:
# # Get and set API key
# with open('C:/Users/ijyli/Documents/OpenAI/anlp23-project.txt', 'r') as file:
#     api_key = file.read()
# openai.api_key = api_key


In [36]:

# Get and set API key
with open('C:/Users/ijyli/Documents/OpenAI/anlp23-project.txt', 'r') as file:
    my_api_key = file.read()

from openai import OpenAI
client = OpenAI(
    api_key="YOUR_API_KEY",
)


NameError: name 'OpenAI' is not defined

Upload Training and Validation File

In [31]:
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)


NameError: name 'client' is not defined

Create Finetuning Job

In [ ]:
suffix_name = this_nb

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response


FineTuningJob(id='ftjob-dyySU8thbv5LBSdq1KrXZiki', created_at=1699779814, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-GopHvIe9DrH2zt9iInyLCoD8', result_files=[], status='validating_files', trained_tokens=None, training_file='file-lH871kh5kCsKUV31vYjbxe7C', validation_file='file-jg2GAagSKpGN7mgicEiKdayI')

### All Finetuning  Jobs

In [ ]:
client.fine_tuning.jobs.list(limit=10)


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-dyySU8thbv5LBSdq1KrXZiki', created_at=1699779814, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-GopHvIe9DrH2zt9iInyLCoD8', result_files=[], status='running', trained_tokens=None, training_file='file-lH871kh5kCsKUV31vYjbxe7C', validation_file='file-jg2GAagSKpGN7mgicEiKdayI'), FineTuningJob(id='ftjob-h5P9gmmDXysjbOD99jz8TtYv', created_at=1699776576, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:futuresmart-ai:yt:8Jzz2CMc', finished_at=1699777187, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-GopHvIe9DrH2zt9iInyLCoD8', result_files=['file-94bTw4zTuPmEg5cDx8UbIg7S'], status='succeeded', trained_tokens=9123, training_file='file-xWkwYEZgnD9boGLZAR

### Retrieve Specific Job

In [ ]:
#response = client.fine_tuning.jobs.retrieve("ftjob-h5P9gmmDXysjbOD99jz8TtYv")
response


FineTuningJob(id='ftjob-h5P9gmmDXysjbOD99jz8TtYv', created_at=1699776576, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:futuresmart-ai:yt:8Jzz2CMc', finished_at=1699777187, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-GopHvIe9DrH2zt9iInyLCoD8', result_files=['file-94bTw4zTuPmEg5cDx8UbIg7S'], status='succeeded', trained_tokens=9123, training_file='file-xWkwYEZgnD9boGLZAR6ebpYO', validation_file='file-R4ZUTOca4nq1l3yzEhDIkbXR')

In [ ]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)



Fine-tuned model id: ft:gpt-3.5-turbo-0613:futuresmart-ai:yt:8Jzz2CMc


### Test Finetuned Model

In [ ]:
def format_test(row):

    formatted_message = [
        {
            "role": "user", 
            "content": row['response'] + "\nYour Task: Rate the coherence of the passage above on a scale of 1 to 10, 1 being incoherent and 10 being very coherent."
        }
    ]
    return formatted_message


def predict(test_messages, fine_tuned_model_id):

    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )

    return response.choices[0].message.content


In [ ]:
def store_predictions(test_df, fine_tuned_model_id):

    print("fine_tuned_model_id",fine_tuned_model_id)
    test_df['Prediction'] = None

    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions_first_50.csv")


In [ ]:
test_df = this_notebook_predictions
store_predictions(test_df, fine_tuned_model_id)


fine_tuned_model_id ft:gpt-3.5-turbo-0613:futuresmart-ai:yt:8Jzz2CMc
